## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.tipo = tipo
        self.cantidad = cantidad

    def valorTotal(self):
        return self.precio * self.cantidad

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if valor >= 0:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if valor >= 0:
            self._cantidad = valor


In [2]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
class Oferta:
    def __init__(self, descripcion, codigo=None, tipos=None):
        self.descripcion = descripcion
        self.codigo = codigo if codigo else []  
        self.tipos = tipos if tipos else []       

    def es_aplicable(self, producto):
        return producto.codigo in self.codigo or producto.tipo in self.tipos


class OfertaDescuento:
    def __init__(self, porcentaje, codigo=None, codigos=None, tipo=None, tipos=None):
        self.porcentaje = porcentaje
        
        self.codigos = codigos if codigos is not None else []
        if codigo:
            
            if isinstance(codigo, list):
                self.codigos.extend(codigo)  
            else:
                self.codigos.append(codigo)  
        self.tipos = tipos if tipos is not None else []
        if tipo:
            self.tipos.append(tipo)
        self.descripcion = f"Descuento del {self.porcentaje}%"

    def es_aplicable(self, producto):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def calcularDescuento(self, producto, cantidad):
        if self.es_aplicable(producto):
            return round(producto.precio * cantidad * (self.porcentaje / 100), 2)
        return 0


class Oferta2x1:
    def __init__(self, tipos=None, codigos=None):
        self.tipos = tipos if tipos else []
        self.codigos = codigos if codigos else []
        self.descripcion = "Oferta 2x1"
    
    def es_aplicable(self, producto):
        return producto.tipo in self.tipos or producto.codigo in self.codigos

    def calcularDescuento(self, producto, cantidad):
        if self.es_aplicable(producto):
            return (cantidad // 2) * producto.precio
        return 0


In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def calcularDescuento(self, producto, cantidad):
        max_descuento = 0
        for oferta in self.ofertas:
            if oferta.es_aplicable(producto):
                descuento = oferta.calcularDescuento(producto, cantidad)
                max_descuento = max(max_descuento, descuento)
        return max_descuento

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(prod.cantidad for prod in self.productos)

    @property
    def valorTotal(self):
        return sum(prod.precio * prod.cantidad for prod in self.productos)

    @property
    def precioPromedio(self):
        if self.cantidadUnidades == 0:
            return 0
        return self.valorTotal / self.cantidadUnidades

    def vender(self, producto, cantidad):
        for prod in self.productos:
            if prod.codigo == producto.codigo:
                if cantidad <= prod.cantidad:
                    descuento = self.calcularDescuento(prod, cantidad)
                    prod.cantidad -= cantidad
                    return prod.precio * cantidad - descuento
                else:
                    raise ValueError("Cantidad insuficiente en stock.")
        raise ValueError("Producto no encontrado en el catálogo.")

    def informe(self):
        informe = []
        informe.append("INFORME CATALOGO")
        informe.append(f"Cantidad de productos:   {self.cantidadProductos}")
        informe.append(f"Cantidad de unidades:    {self.cantidadUnidades}")
        
        informe.append(f"Precio Promedio:       $ {self.precioPromedio:.2f}")
        informe.append(f"Valor total:           $ {self.valorTotal:.2f}")
        
        tipos = {}
        for producto in self.productos:
            if producto.tipo not in tipos:
                tipos[producto.tipo] = {'unidades': 0, 'total': 0}
            tipos[producto.tipo]['unidades'] += producto.cantidad
            tipos[producto.tipo]['total'] += producto.precio * producto.cantidad
        
        informe.append("Tipos de productos: ")
        for tipo, datos in tipos.items():
            precio_promedio_tipo = datos['total'] / datos['unidades'] if datos['unidades'] > 0 else 0
            informe.append(f"  - {tipo:<20}:  {datos['unidades']}u x $ {precio_promedio_tipo:.2f}")

        informe.append("Ofertas:")
        for oferta in self.ofertas:
            informe.append(f"  - {oferta.descripcion}")
        
        return "\n".join(informe)

    def guardar(self, nombre_archivo):
        with open(nombre_archivo, mode='w', newline='') as archivo:
            writer = csv.writer(archivo)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])

    @classmethod
    def leer(cls, nombre_archivo):
        catalogo = cls()
        with open(nombre_archivo, mode='r') as archivo:
            reader = csv.DictReader(archivo)
            for fila in reader:
                codigo = fila['codigo']
                nombre = fila['nombre']
                precio = float(fila['precio'])
                tipo = fila['tipo']
                cantidad = int(fila['cantidad'])
                producto = Producto(codigo, nombre, precio, tipo, cantidad)
                catalogo.agregar(producto)
        return catalogo


catalogo = Catalogo.leer('catalogo.csv')


o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

print(catalogo.informe())



INFORME CATALOGO
Cantidad de productos:   30
Cantidad de unidades:    1000
Precio Promedio:       $ 2000.00
Valor total:           $ 2000000.00
Tipos de productos: 
  - gaseosa             :  120u x $ 1399.17
  - galleta             :  255u x $ 2165.10
  - fideo               :  125u x $ 2096.00
  - yogur               :  150u x $ 2256.67
  - agua                :  150u x $ 2933.33
  - arroz               :  200u x $ 1197.50
Ofertas:
  - Oferta 2x1
  - Descuento del 20%
  - Descuento del 10%


In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
class Cliente:
    def __init__(self, nombre, cuit):
        self.nombre = nombre
        self.cuit = cuit
    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if valor:
            self._nombre = valor

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        if self.validar_cuit(valor):
            self._cuit = valor

    def validar_cuit(self, cuit):
        if len(cuit) == 13 and cuit[2] == '-' and cuit[11] == '-':
            return (cuit[:2].isdigit() and 
                    cuit[3:11].isdigit() and 
                    cuit[12].isdigit())
        return False


In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [9]:
from datetime import datetime

class Factura:
    numero_actual = 100

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.now().strftime("%d/%m/%Y")
        self.productos_vendidos = {}

    @classmethod
    def nuevoNumero(cls):
        cls.numero_actual += 1
        return cls.numero_actual

    @classmethod
    def ultimaFactura(cls, numero):
        cls.numero_actual = numero

    def agregar(self, producto, cantidad):
        if not producto:
            raise ValueError("Producto no encontrado")
        
        producto_catalogo = self.catalogo.buscar(producto.codigo)
        if not producto_catalogo or producto_catalogo.cantidad < cantidad:
            raise ValueError("No hay suficiente stock")
        
        producto_catalogo.cantidad -= cantidad
        
        if producto.codigo in self.productos_vendidos:
            self.productos_vendidos[producto.codigo]['cantidad'] += cantidad
        else:
            self.productos_vendidos[producto.codigo] = {'producto': producto, 'cantidad': cantidad}

    @property
    def cantidadProductos(self):
        return len(self.productos_vendidos)

    @property
    def cantidadUnidades(self):
        return sum(item['cantidad'] for item in self.productos_vendidos.values())

    @property
    def subtotal(self):
        return sum(item['producto'].precio * item['cantidad'] for item in self.productos_vendidos.values())

    @property
    def descuentos(self):
        total_descuento = 0
        for codigo, item in self.productos_vendidos.items():
            producto = item['producto']
            cantidad = item['cantidad']
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            total_descuento += descuento
        return total_descuento

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        lineas = []
        lineas.append(f"Factura: {self.numero}")
        lineas.append(f"Fecha  : {self.fecha}")
        lineas.append(f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})")
        lineas.append("")  

        for codigo, item in self.productos_vendidos.items():
            producto = item['producto']
            cantidad = item['cantidad']
            subtotal_producto = producto.precio * cantidad
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            
            lineas.append(f"{cantidad} {producto.nombre:<15} x ${producto.precio:>7.2f} = ${subtotal_producto:>7.2f}")

            if descuento > 0:
                oferta_aplicable = next((oferta for oferta in self.catalogo.ofertas if oferta.es_aplicable(producto)), None)
                if oferta_aplicable:
                    if isinstance(oferta_aplicable, OfertaDescuento):
                        lineas.append(f"      Descuento {oferta_aplicable.porcentaje}%          - ${descuento:>7.2f}")
                    elif isinstance(oferta_aplicable, Oferta2x1):
                        lineas.append(f"      Oferta 2x1              - ${descuento:>7.2f}")

        lineas.append("") 
        lineas.append(f"                                             Subtotal:   ${self.subtotal:>7.2f}")
        lineas.append(f"                                             Descuentos: ${self.descuentos:>7.2f}")
        lineas.append(f"                                             -----------------------")
        lineas.append(f"                                             TOTAL:      ${self.total:>7.2f}")

        factura_impresa = "\n".join(lineas)
        print(factura_impresa)  
        return factura_impresa

In [10]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Factura: 103
Fecha  : 05/10/2024
Cliente: Juan Perez (20-12345678-9)

10 Coca Cola       x $1500.00 = $15000.00
      Descuento 10%          - $1500.00
3 Sonrisa         x $1200.00 = $3600.00
      Oferta 2x1              - $1200.00

                                             Subtotal:   $18600.00
                                             Descuentos: $2700.00
                                             -----------------------
                                             TOTAL:      $15900.00
Prueba pasada exitosamente!


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
